In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import re
from importlib import reload

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer as Imputer
from sklearn.metrics import mean_squared_log_error

from sklearn.linear_model import LinearRegression

from _lib.preprocess import preprocess_missing as prep
from _lib.preprocess import get_instruction as info
from _lib.preprocess_test import preprocess_missing as prep_test
from _lib.create_output import output

df = pd.read_csv("_database/Input/train.csv", index_col = 0)

df = prep(df)
df = df.drop([1299, 935, 186, 347, 1231, 1183, 692, 955])
X_data = df.drop("SalePrice", axis = 1)
y = df["SalePrice"]




In [55]:
# Function to generate polynomial features of numerical variables
def gen_poly(data, degree):
    '''
    Input : Vector or matrix 
    
    Return matrix of polynomial for each polynomial degree from 1 to degree calculated on each column
    '''
    
    result = np.concatenate([np.power(data, d) for d in np.arange(1, degree + 1)], axis = 1)
    return result

df_type = pd.DataFrame({"Columns" : X_data.columns, "Type" : [str(X_data[col].dtype) for col in X_data.columns]})
num_columns = df_type.loc[(df_type["Type"] == "int64") | (df_type["Type"] == "float64")]["Columns"]
cat_columns = df_type.loc[(df_type["Type"] == "category")]["Columns"]
bool_columns = df_type.loc[df_type["Type"] == "bool"]["Columns"]

# Categories in categorical features
list_categories = [np.array(info(col)) for col in cat_columns.values]
list_categories[14] = np.arange(1, 11)
list_categories[15] = np.arange(1, 11)


# Polynomial degree
poly_degree = 1

get_numerical = FunctionTransformer(lambda x : x[num_columns.values].values, validate = False)

get_category = FunctionTransformer(lambda x : x[cat_columns.values], validate = False)

get_bool = FunctionTransformer(lambda x : x[bool_columns.values].values, validate = False)

generate_poly = FunctionTransformer(lambda x : gen_poly(x, poly_degree), validate = False)

pipeline_num_prep = Pipeline([('selector', get_numerical),
                              ('poly', generate_poly)])

pipeline_cat_prep = Pipeline([('selector', get_category),
                              ('Dummy', OneHotEncoder(drop = 'first', sparse = False,
                                                     categories = list_categories))])

transformers = [ ('Numerical', pipeline_num_prep), ('Categorical', pipeline_cat_prep), ('Bool', get_bool) ]

preprocess_union = FeatureUnion(transformer_list = transformers)

pl = Pipeline([
    ('union', preprocess_union)

])

In [56]:
X = pl.fit_transform(X_data)

In [57]:
y = np.log(y)

Split the data 

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 43)

In [ ]:
ml = LinearRegression()

In [ ]:
ml.fit(X_train, y_train)

In [ ]:
y_pred = ml.predict(X_test)

In [ ]:
msle = (np.sum((np.log(y_pred + 1) - np.log(y_test + 1))**2))/(len(y_pred))

In [ ]:
print(msle)

In [ ]:
mean_squared_error(y_test, y_pred)

In [ ]:
mean_squared_log_error(y_test[y_pred > 0], y_pred[y_pred > 0])

## Using poly degree > 1

In [ ]:
def poly_reg_pipeline(poly_degree):

    get_numerical = FunctionTransformer(lambda x : x[num_columns.values].values, validate = False)

    get_category = FunctionTransformer(lambda x : x[cat_columns.values], validate = False)

    get_bool = FunctionTransformer(lambda x : x[bool_columns.values].values, validate = False)

    generate_poly = FunctionTransformer(lambda x : gen_poly(x, poly_degree), validate = False)

    pipeline_num_prep = Pipeline([('selector', get_numerical),
                                  ('poly', generate_poly)])

    pipeline_cat_prep = Pipeline([('selector', get_category),
                                  ('Dummy', OneHotEncoder(drop = 'first', sparse = False,
                                                         categories = list_categories))])

    transformers = [ ('Numerical', pipeline_num_prep), ('Categorical', pipeline_cat_prep), ('Bool', get_bool) ]

    preprocess_union = FeatureUnion(transformer_list = transformers)

    pl = Pipeline([
        ('union', preprocess_union)

    ])
    
    return pl

In [ ]:
n = 20
seed = 40
msle_list = []
for i in range(1, n):
    pl = poly_reg_pipeline(i)
    X = pl.fit_transform(X_data)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = seed)
    ml = LinearRegression()
    ml.fit(X_train, y_train)
    
    y_pred = ml.predict(X_test)
    msle = mean_squared_error(y_test, y_pred)
    msle_list.append(msle)
    
    print("The MSLE for poly degree {} is : {}".format(i, msle))

Next we want to compare them using standard scaler

## Ridge

In [12]:
from sklearn.linear_model import Ridge

In [ ]:
pl = poly_reg_pipeline(3)
X = pl.fit_transform(X_data)
y1 = df["SalePrice"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 40)

In [ ]:
alpha_list = np.logspace(-3, 1, 50)

In [ ]:
mse = np.zeros(len(alpha_list))
for i in np.arange(len(alpha_list)):
    ml = Ridge(alpha = alpha_list[i])
    ml.fit(X_train, y_train)
    
    y_pred = ml.predict(X_test)
    
    mse[i] = mean_squared_error(y_test, y_pred)
    

In [ ]:
fig, ax = plt.subplots()
ax.plot(alpha_list, mse)

In [29]:
alpha_list1 = np.logspace(-3, 2, 50)
alpha_list2 = np.array([0, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100, 300, 1000])
alpha_list = np.concatenate((alpha_list1, alpha_list2))

In [ ]:
ml = Ridge()

In [ ]:
params = {'alpha' : alpha_list}

In [ ]:
cv = GridSearchCV(ml, param_grid = params, cv = 10)

In [ ]:
cv.fit(X_train, y_train)

In [ ]:
cv.best_params_

In [ ]:
ml = cv.best_estimator_

In [ ]:
y_pred = ml.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred)

#### Using Bootstrap

In [ ]:
def bootstrap_ml(x, y, ml, size = 1):
    inds = np.arange(x.shape[0])
    bs_rep = np.empty(size)
    
    for i in range(size):
        
        bs_inds = np.random.choice(inds, size = len(inds))
        x_bs = x[bs_inds]
        y_bs = y.values.reshape(-1,1)[bs_inds]
        ml.fit(x_bs, y_bs)
        
        y_bs_pred = ml.predict(X_test)
        
        bs_rep[i] = mean_squared_error(y_test, y_bs_pred)
        
    return bs_rep

In [ ]:
bss = bootstrap_ml(X_train, y_train, ml, size = 100)

In [ ]:
plt.hist(bss[bss < 0.02], bins = 30)

## another polynomial degree

In [ ]:
ml = Ridge()

In [ ]:
pl = poly_reg_pipeline(5)
X = pl.fit_transform(X_data)
y1 = df["SalePrice"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 40)

In [ ]:
cv = GridSearchCV(ml, param_grid = params, cv = 10)

In [ ]:
cv.fit(X_train, y_train)

In [ ]:
ridge_param = cv.best_params_

In [ ]:
ridge_param

In [ ]:
ml = cv.best_estimator_

In [ ]:
y_pred = ml.predict(X_test)
y_pred_train = ml.predict(X_train)

In [ ]:
mean_squared_error(y_test, y_pred)

In [ ]:
mean_squared_error(y_train, y_pred_train)

## Lasso

In [10]:
from sklearn.linear_model import Lasso

In [ ]:
ml = Lasso()

In [ ]:
cv = GridSearchCV(ml, param_grid = params, cv = 10)

In [ ]:
cv.fit(X_train, y_train)

In [ ]:
lasso_param = cv.best_params_

In [ ]:
ml = cv.best_estimator_

In [ ]:
y_pred = ml.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred)

## Random Forest

In [11]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
DecisionTreeRegressor()

In [ ]:
RandomForestRegressor()

In [ ]:
params = {'min_samples_leaf' : np.linspace(0.1, 1, 50),
         'max_depth' : np.linspace(1, 300, 100)}

In [ ]:
ml = DecisionTreeRegressor()

In [ ]:
cv = GridSearchCV(ml, param_grid = params, cv = 10, n_jobs=-1, verbose = 1)

In [ ]:
cv.fit(X_train, y_train)

In [ ]:
cv.best_params_

In [ ]:
ml = cv.best_estimator_

In [ ]:
y_pred = ml.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred)

In [ ]:
ml = RandomForestRegressor()

In [ ]:
params = {'min_samples_leaf' : np.linspace(0.0001, 0.001, 10),
         'max_depth' : np.linspace(130, 140, 5),
         'n_estimators' : np.arange(33, 39)}

In [ ]:
cv = GridSearchCV(ml, param_grid = params, n_jobs=-1, verbose = 2)

In [ ]:
cv.fit(X_train, y_train)

In [ ]:
cv.best_params_

In [ ]:
ml = cv.best_estimator_

In [ ]:
y_pred = ml.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred)

## Boosting

In [60]:
from sklearn.ensemble import GradientBoostingRegressor

In [61]:
GradientBoostingRegressor()

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [62]:
ml = GradientBoostingRegressor()

In [63]:
params = {'alpha' : np.linspace(0.5, 0.7, 5),
          'max_depth' : np.arange(1, 4),
         'max_features' : np.linspace(0.2, 0.6, 10),
         'n_estimators' : np.linspace(60, 80, 10),
         'subsample' : np.linspace(0.4, 0.6, 10)}

In [64]:
cv = GridSearchCV(ml, param_grid = params, n_jobs=-1, verbose = 5)

In [65]:
cv.fit(X, y)

Fitting 5 folds for each of 15000 candidates, totalling 75000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 492 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 2508 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 5100 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 8268 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done 12012 tasks      | elapsed:   18.2s
[Parallel(n_jobs=-1)]: Done 16332 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done 21228 tasks      | elapsed:   31.1s
[Parallel(n_jobs=-1)]: Done 26700 tasks      | elapsed:   38.0s
[Parallel(n_jobs=-1)]: Done 32748 tasks      | elapsed:   46.2s
[Parallel(n_jobs=-1)]: Done 39372 tasks      | elapsed:   55.6s
[Parallel(n_jobs=-1)]: Done 46572 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 54348 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 62700 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 71628 

TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [ ]:
cv.best_params_

In [ ]:
ml = GradientBoostingRegressor(alpha = 0.5, max_depth = 1, max_features = 0.2, n_estimators=60, subsample = 0.4)

In [ ]:
ml.fit(X_train, y_train)

In [ ]:
y_pred = ml.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred)

### combined

In [15]:
combined_pred = np.zeros(len(y_test))
mses = np.zeros(len(regressors) +1)

for i in np.arange(len(regressors)):
    print("Working on regressor {}".format(i+1))
    reg = regressors[i]
    reg.fit(X_train, y_train)
    
    y_pred = reg.predict(X_test)
    mses[i] = mean_squared_error(y_test, y_pred)
    combined_pred = combined_pred + y_pred

combined_pred = combined_pred / len(regressors)
mses[len(regressors)] = mean_squared_error(y_test, combined_pred)

NameError: name 'y_test' is not defined

In [32]:
regressors = [LinearRegression(), Ridge(), Lasso(),
              RandomForestRegressor(),
              GradientBoostingRegressor()]

In [41]:
params_linreg = {'normalize' : [True, False]}
params_ridgelasso = {'alpha' : alpha_list}
params_rf = {'min_samples_leaf' : np.linspace(0.0001, 0.001, 10),
         'max_depth' : np.linspace(130, 140, 5),
         'n_estimators' : np.arange(33, 39)}
params_sgbr = {'alpha' : np.linspace(0.5, 0.7, 5),
          'max_depth' : np.arange(1, 4),
         'max_features' : np.linspace(0.2, 0.6, 10),
         'n_estimators' : np.linspace(60, 80, 10),
         'subsample' : np.linspace(0.4, 0.6, 10)}
params_list = [params_linreg, params_ridgelasso, params_ridgelasso, params_rf, params_sgbr]

In [42]:
combined_pred = np.zeros(X_test.shape[0])

for i in np.arange(len(regressors)-1):
    reg = regressors[i]
    print("Working on {}".format(reg))
    params = params_list[i]
    
    cv = GridSearchCV(reg, param_grid = params, cv = 10, n_jobs=-1, verbose = 3)
    cv.fit(X, y)
    print("The best parameters are : {}".format(cv.best_params_))
    
    ml = cv.best_estimator_
    y_pred = ml.predict(X_test)
    combined_pred = combined_pred + y_pred


Working on LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
Fitting 10 folds for each of 2 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  20 | elapsed:    1.5s remaining:    6.2s
[Parallel(n_jobs=-1)]: Done  11 out of  20 | elapsed:    1.5s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done  18 out of  20 | elapsed:    1.6s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    1.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


The best parameters are : {'normalize': False}
Working on Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
Fitting 10 folds for each of 64 candidates, totalling 640 fits


[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 296 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 617 out of 640 | elapsed:    2.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 640 out of 640 | elapsed:    2.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


The best parameters are : {'alpha': 7.543120063354623}
Working on Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)
Fitting 10 folds for each of 64 candidates, totalling 640 fits


[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 360 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done 617 out of 640 | elapsed:   15.4s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done 640 out of 640 | elapsed:   15.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


The best parameters are : {'alpha': 0.001}
Working on RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)
Fitting 10 folds for each of 300 candidates, totalling 3000 fits


[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done 264 tasks      | elapsed:   33.8s
[Parallel(n_jobs=-1)]: Done 488 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 1128 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 1544 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 2024 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 2568 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 3000 out of 3000 | elapsed:  6.2min finished


The best parameters are : {'max_depth': 130.0, 'min_samples_leaf': 0.0005, 'n_estimators': 34}


In [40]:
sgbr_best = cv.best_params_

In [43]:
ml = GradientBoostingRegressor(alpha = 0.5, max_depth = 1, max_features = 0.2, n_estimators = 60, subsample = 0.4)

In [44]:
ml.fit(X, y)
y_pred = ml.predict(X_test)
combined_pred = combined_pred + y_pred

In [45]:
combined_pred = np.exp(combined_pred / len(regressors))

## Prediction

In [16]:
test = pd.read_csv("_database/Input/test.csv", index_col = 0)
test_index = test.index

In [17]:
X_test = prep_test(test)

In [18]:
X_test = pl.transform(X_test)

In [ ]:
regressors = [LinearRegression(), Ridge(), Lasso(),
              RandomForestRegressor(),
              GradientBoostingRegressor()]

In [ ]:
params_linreg = {'normalize' : [True, False]}
params_ridgelasso = {'alpha' : alpha_list}
params_rf = {'min_samples_leaf' : np.linspace(0.0001, 0.001, 10),
         'max_depth' : np.linspace(130, 140, 5),
         'n_estimators' : np.arange(33, 39)}
params_sgbr = {'alpha' : np.linspace(0.5, 0.7, 5),
          'max_depth' : np.arange(1, 4),
         'max_features' : np.linspace(0.2, 0.6, 10),
         'n_estimators' : np.linspace(60, 80, 10),
         'subsample' : np.linspace(0.4, 0.6, 10)}
params_list = [params_linreg, params_ridgelasso, params_ridgelasso, params_rf, params_sgbr]

In [25]:
combined_pred = np.zeros(X_test.shape[0])

for i in np.arange(len(regressors)-1):
    reg = regressors[i]
    print("Working on {}".format(reg))
    params = params_list[i]
    
    cv = GridSearchCV(reg, param_grid = params, cv = 10, n_jobs=-1, verbose = 3)
    cv.fit(X, y)
    print("The best parameters are : {}".format(cv.best_params_))
    
    ml = cv.best_estimator_
    y_pred = ml.predict(X_test)
    combined_pred = combined_pred + y_pred


Working on regressor 1
Working on regressor 2
Working on regressor 3
Working on regressor 4
Working on regressor 5


In [ ]:
ml = GradientBoostingRegressor(alpha = 0.5, max_depth = 1, max_features = 0.2, n_estimators = 60, subsample = 0.4)

In [ ]:
ml.fit(X, y)
y_pred = ml.predict(X_test)
combined_pred = combined_pred + y_pred
combined_pred = np.exp(combined_pred / len(regressors))

In [47]:
pred = pd.DataFrame({"id" : test_index,
             "SalePrice" : combined_pred})

In [48]:
pred = pred.set_index("id")

In [49]:
pred.to_csv("_database/Output/Combined.csv")